In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [2]:
crashes = pd.read_csv("uncleaned_data.csv", parse_dates=[['crash_date', 'crash_time']]) # keeping the original file as backup
df = crashes
print("Original size: ", df.shape)
# Displaying the 5 first rows to discover the dataset
# transposing to be able see everything on the screen
df.head().T

Original size:  (100000, 28)


,0,1,2,3,4
crash_date_crash_time,2017-04-18 23:10:00,2017-05-06 13:00:00,2017-04-27 17:15:00,2017-05-09 20:10:00,2017-04-18 14:00:00
borough,STATEN ISLAND,BRONX,QUEENS,NaN,BRONX
zip_code,10312.0,10472.0,11420.0,NaN,10456.0
latitude,40.536728,40.829052,40.677303,40.624958,40.828846
longitude,-74.193344,-73.85038,-73.804565,-74.145775,-73.90312
location,"(40.536728, -74.193344)","(40.829052, -73.85038)","(40.677303, -73.804565)","(40.624958, -74.145775)","(40.828846, -73.90312)"
on_street_name,NaN,CASTLE HILL AVENUE,135 STREET,FOREST AVENUE,NaN
off_street_name,NaN,BLACKROCK AVENUE,FOCH BOULEVARD,RICHMOND AVENUE,NaN
cross_street_name,243 DARLINGTON AVENUE,NaN,NaN,NaN,1167 BOSTON ROAD
number_of_persons_injured,0,1,0,1,0


In [3]:
# Check for empty rows (row with missing values in all columns)
# Having at least one empty row means that all columns would have at least one missing value
df.isna().sum()

crash_date_crash_time                0
borough                          35026
zip_code                         35034
latitude                          8035
longitude                         8035
location                          8035
on_street_name                   26009
off_street_name                  52875
cross_street_name                74033
number_of_persons_injured            0
number_of_persons_killed             0
number_of_pedestrians_injured        0
number_of_pedestrians_killed         0
number_of_cyclist_injured            0
number_of_cyclist_killed             0
number_of_motorist_injured           0
number_of_motorist_killed            0
contributing_factor_vehicle_1      371
contributing_factor_vehicle_2    19243
contributing_factor_vehicle_3    91239
contributing_factor_vehicle_4    97760
contributing_factor_vehicle_5    99333
collision_id                         0
vehicle_type_code1                 740
vehicle_type_code2               26589
vehicle_type_code_3      

=> 11 columns don't have any missing values. So there is not one single empty row.

In [4]:
# Check for duplicated rows
df.duplicated().any()

False

In [5]:
# Dropping columns
df.drop(['location','borough','off_street_name','collision_id'], axis=1, inplace=True)

In [6]:
# Data format is correct
df.dtypes

crash_date_crash_time            datetime64[ns]
zip_code                                float64
latitude                                float64
longitude                               float64
on_street_name                           object
cross_street_name                        object
number_of_persons_injured                 int64
number_of_persons_killed                  int64
number_of_pedestrians_injured             int64
number_of_pedestrians_killed              int64
number_of_cyclist_injured                 int64
number_of_cyclist_killed                  int64
number_of_motorist_injured                int64
number_of_motorist_killed                 int64
contributing_factor_vehicle_1            object
contributing_factor_vehicle_2            object
contributing_factor_vehicle_3            object
contributing_factor_vehicle_4            object
contributing_factor_vehicle_5            object
vehicle_type_code1                       object
vehicle_type_code2                      

In [7]:
# renaming column
new_columns = {'crash_date_crash_time': 'crash_datetime',
               'vehicle_type_code1': 'vehicle_type_code_1',
               'vehicle_type_code1': 'vehicle_type_code_1'}
df.rename(columns=new_columns, inplace=True)
df.columns

Index(['crash_datetime', 'zip_code', 'latitude', 'longitude', 'on_street_name',
       'cross_street_name', 'number_of_persons_injured',
       'number_of_persons_killed', 'number_of_pedestrians_injured',
       'number_of_pedestrians_killed', 'number_of_cyclist_injured',
       'number_of_cyclist_killed', 'number_of_motorist_injured',
       'number_of_motorist_killed', 'contributing_factor_vehicle_1',
       'contributing_factor_vehicle_2', 'contributing_factor_vehicle_3',
       'contributing_factor_vehicle_4', 'contributing_factor_vehicle_5',
       'vehicle_type_code_1', 'vehicle_type_code2', 'vehicle_type_code_3',
       'vehicle_type_code_4', 'vehicle_type_code_5'],
      dtype='object')

In [8]:
def narrowing_down_factor(factor):
    distraction = ['Driver Inattention/Distraction','Passenger Distraction','Fell Asleep','Outside Car Distraction',
                   'Fatigued/Drowsy','Cell Phone (hand-Held)','Using On Board Navigation Device',
                   'Tinted Windows','Eating or Drinking', 'Other Electronic Device', 'Cell Phone (hands-free)',
                   'Listening/Using Headphones']
    driver_mistake = ['Following Too Closely','Failure to Yield Right-of-Way','Backing Unsafely',
                      'Passing or Lane Usage Improper','Passing Too Closely','Unsafe Lane Changing',
                      'Turning Improperly','Driver Inexperience', 'Failure to Keep Right',
                      'Driverless/Runaway Vehicle', 'Oversized Vehicle']
    illegal_action = ['Unsafe Speed','Alcohol Involvement','Traffic Control Disregarded','Aggressive Driving/Road Rage',
                      'Drugs (illegal)']
    other_involvement =  ['Other Vehicular','Reaction to Uninvolved Vehicle', 'Animals Action',
                          'View Obstructed/Limited','Pedestrian/Bicyclist/Other Pedestrian Error/Confusion',
                           'Glare','Vehicle Vandalism']
    bad_road_infrastructure = ['Pavement Slippery', 'Obstruction/Debris','Pavement Defective',
                               'Other Lighting Defects']
    car_failures = ['Brakes Defective', 'Steering Failure', 'Tire Failure/Inadequate',
                    'Traffic Control Device Improper/Non-Working', 'Lane Marking Improper/Inadequate',
                    'Tow Hitch Defective','Headlights Defective', 'Shoulders Defective/Improper',
                    'Windshield Inadequate']
    medical = ['Lost Consciousness', 'Illnes','Accelerator Defective','Physical Disability','Prescription Medication']
    if factor in medical:
        return "medical"
    elif factor in car_failures:
        return "car_failure"
    elif factor in bad_road_infrastructure:
        return "bad_road_infrastructure"
    elif factor in other_involvement:
        return "other_involvement"
    elif factor in illegal_action:
        return "illegal_action"
    elif factor in driver_mistake:
        return "driver_mistake"
    elif factor in distraction:
        return "distraction"

In [9]:
df['factor_1'] = df['contributing_factor_vehicle_1'].apply(lambda x: narrowing_down_factor(x))
df['factor_2'] = df['contributing_factor_vehicle_2'].apply(lambda x: narrowing_down_factor(x))
df['factor_3'] = df['contributing_factor_vehicle_3'].apply(lambda x: narrowing_down_factor(x))
df['factor_4'] = df['contributing_factor_vehicle_4'].apply(lambda x: narrowing_down_factor(x))
df['factor_5'] = df['contributing_factor_vehicle_5'].apply(lambda x: narrowing_down_factor(x))

In [10]:
df.head()

,crash_datetime,zip_code,latitude,longitude,on_street_name,cross_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,...,vehicle_type_code_1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,factor_1,factor_2,factor_3,factor_4,factor_5
0,2017-04-18 23:10:00,10312.0,40.536728,-74.193344,NaN,243 DARLINGTON AVENUE,0,0,0,0,...,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,distraction,None,None,None,None
1,2017-05-06 13:00:00,10472.0,40.829052,-73.850380,CASTLE HILL AVENUE,NaN,1,0,1,0,...,Sedan,NaN,NaN,NaN,NaN,driver_mistake,None,None,None,None
2,2017-04-27 17:15:00,11420.0,40.677303,-73.804565,135 STREET,NaN,0,0,0,0,...,Sedan,Sedan,NaN,NaN,NaN,distraction,None,None,None,None
3,2017-05-09 20:10:00,NaN,40.624958,-74.145775,FOREST AVENUE,NaN,1,0,0,0,...,Motorcycle,Sedan,Bus,NaN,NaN,None,None,None,None,None
4,2017-04-18 14:00:00,10456.0,40.828846,-73.903120,NaN,1167 BOSTON ROAD,0,0,0,0,...,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,distraction,None,None,None,None


In [11]:
for col in ['factor_1','factor_2','factor_3','factor_4','factor_5']:
    df = pd.get_dummies(df[col], prefix=col, drop_first="True")
    df.rename(columns={col+"_yes" : col})

df.head().T

KeyError: 'factor_2'